In [5]:
# 计算 y(x)= x^2 在x=3时的导数, y=a*w
import tensorflow as tf
a = tf.constant(1.)
b = tf.constant(2.)
c = tf.constant(3.)


w=tf.Variable(initial_value=4.)  # 声明变量, 变量赋值初始化
with tf.GradientTape() as tape:
    y= a*tf.square(w)+b*w+c
y_grad = tape.gradient(y,w)
y_grad   #注意，张量与变量的区别

# 变量与普通张量的一个重要区别是其默认能够被 TensorFlow 的自动求导机制所求导，
# 因此往往被用于定义机器学习模型的参数。

5.660050366720203
-3.205236284431745
4.345955160759873
-2.737828369173383
0.2783681410287979
4.961270066161866
1.761601771981681
-4.6150373894773145
9.916469851803235
7.18767954718443
1.3918477777320692
2.9594067617427946
1.4101442536818674
-6.377210702374255
-2.6848799166687654
5.583101442906349
-9.757415183775333
-1.2861249481116204
0.27740864232258566
1.8568942355910334
-0.9148996641754881
8.574293459943217
6.380717082834014
-8.969791518515876
4.689545723757496
9.72892023349499
4.793907755841198
5.130209814158453
-8.717482638293692
-2.2258799896707533
-4.962463149766987
-9.138195810665994
-2.1616503426055855
-6.194911988715566
-0.2519134635216247
4.777426675724595
-2.4399108961954568
-8.230055327056444
-5.698351533656298
9.608178896666796
5.632006665201617
-1.487776577894044
-1.0577523268813405
3.6372142395948934
-9.331298582616654
1.5100777893650186
0.8967508835683535
9.146993483634343
-5.627628445804649
-8.399914248424736
4.159366015891415
0.11137157311841683
-1.0744231898117178
6